In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
try:
    df = pd.read_csv('ETF prices.csv')  # File name as provided
except FileNotFoundError:
    print("Error: 'ETF prices.csv' not found. Ensure the file is in the same directory as this script or provide the full path.")
    exit(1)
except pd.errors.EmptyDataError:
    print("Error: 'ETF prices.csv' is empty.")
    exit(1)

# Display column names and first few rows for debugging
print("Dataset columns:", df.columns.tolist())
print("First few rows:\n", df.head())

# Define expected column names (update these based on your dataset)
# Assumptions based on your proposal; adjust as needed
col_mapping = {
    'fund_type': 'fund_type',  # e.g., 'Mutual Fund', 'ETF' (may only be 'ETF' if dataset is ETF-only)
    'category': 'fund_category',  # e.g., 'Large-Cap Equity', 'Bond'
    '5_year_return': '5_year_return',  # Annualized 5-year return (%)
    'beta': 'beta',  # Risk metric
    'expense_ratio': 'expense_ratio',  # Annual expense ratio (%)
    'sharpe_ratio': 'sharpe_ratio',  # Risk-adjusted return
    'total_net_assets': 'total_net_assets'  # Fund size
}

# Check if required columns exist
missing_cols = [col for col in col_mapping.values() if col not in df.columns]
if missing_cols:
    print(f"Warning: Missing columns: {missing_cols}")
    print("Please update col_mapping with correct column names from the printed column list above.")
    # Attempt to proceed with available columns
    available_cols = [col for col in col_mapping.values() if col in df.columns]
    if len(available_cols) < 4:  # Need at least 4 columns for meaningful visualizations
        print("Error: Too few required columns available. Exiting.")
        exit(1)

# Rename columns for consistency
df = df.rename(columns={v: k for k, v in col_mapping.items() if v in df.columns})

# Data cleaning
# Remove rows with missing values in key columns (if they exist)
key_cols = [col for col in ['5_year_return', 'beta', 'expense_ratio', 'sharpe_ratio', 'fund_type', 'category'] if col in df.columns]
if key_cols:
    df = df.dropna(subset=key_cols)
else:
    print("Warning: No key columns available for cleaning. Proceeding with caution.")

# Clip extreme values to handle outliers (adjust ranges based on your data)
if 'expense_ratio' in df.columns:
    df['expense_ratio'] = df['expense_ratio'].clip(0.1, 2.0)
if '5_year_return' in df.columns:
    df['5_year_return'] = df['5_year_return'].clip(-10, 20)
if 'beta' in df.columns:
    df['beta'] = df['beta'].clip(0, 2)
if 'sharpe_ratio' in df.columns:
    df['sharpe_ratio'] = df['sharpe_ratio'].clip(-2, 3)

# Visualization 1: Scatter Plot of Risk (Beta) vs. 5-Year Return
if 'beta' in df.columns and '5_year_return' in df.columns and 'fund_type' in df.columns:
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df, x='beta', y='5_year_return', hue='fund_type', style='fund_type', alpha=0.6)
    plt.title('Risk (Beta) vs. 5-Year Return by Fund Type')
    plt.xlabel('Beta (Risk)')
    plt.ylabel('5-Year Return (%)')
    plt.legend(title='Fund Type')
    plt.grid(True)
    plt.savefig('risk_vs_return.png')
    plt.close()
else:
    print("Warning: Skipping scatter plot due to missing columns: beta, 5_year_return, or fund_type")

# Visualization 2: Bar Chart of Average Expense Ratio by Fund Type
if 'expense_ratio' in df.columns and 'fund_type' in df.columns:
    expense_by_type = df.groupby('fund_type')['expense_ratio'].mean().reset_index()
    plt.figure(figsize=(8, 6))
    sns.barplot(data=expense_by_type, x='fund_type', y='expense_ratio', hue='fund_type')
    plt.title('Average Expense Ratio by Fund Type')
    plt.xlabel('Fund Type')
    plt.ylabel('Expense Ratio (%)')
    plt.grid(True, axis='y')
    plt.savefig('expense_ratio_by_type.png')
    plt.close()
else:
    print("Warning: Skipping bar chart due to missing columns: expense_ratio or fund_type")

# Visualization 3: Histogram of 5-Year Returns
if '5_year_return' in df.columns and 'fund_type' in df.columns:
    plt.figure(figsize=(10, 6))
    sns.histplot(data=df, x='5_year_return', bins=30, kde=True, hue='fund_type', multiple='stack')
    plt.title('Distribution of 5-Year Returns')
    plt.xlabel('5-Year Return (%)')
    plt.ylabel('Count')
    plt.grid(True)
    plt.savefig('returns_histogram.png')
    plt.close()
else:
    print("Warning: Skipping histogram due to missing columns: 5_year_return or fund_type")

# Visualization 4: Box Plot of Sharpe Ratios by Fund Category
if 'sharpe_ratio' in df.columns and 'category' in df.columns:
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=df, x='category', y='sharpe_ratio', hue='category')
    plt.title('Sharpe Ratio Distribution by Fund Category')
    plt.xlabel('Fund Category')
    plt.ylabel('Sharpe Ratio')
    plt.grid(True, axis='y')
    plt.xticks(rotation=15)
    plt.savefig('sharpe_ratio_boxplot.png')
    plt.close()
else:
    print("Warning: Skipping box plot due to missing columns: sharpe_ratio or category")

# Visualization 5: Correlation Heatmap
numerical_cols = [col for col in ['5_year_return', 'beta', 'expense_ratio', 'sharpe_ratio', 'total_net_assets'] if col in df.columns]
if len(numerical_cols) >= 2:
    corr_matrix = df[numerical_cols].corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    plt.title('Correlation Heatmap of Numerical Features')
    plt.savefig('correlation_heatmap.png')
    plt.close()
else:
    print("Warning: Skipping heatmap due to insufficient numerical columns")

print("Visualizations complete. Check for saved files: risk_vs_return.png, expense_ratio_by_type.png, returns_histogram.png, sharpe_ratio_boxplot.png, correlation_heatmap.png")

Dataset columns: ['fund_symbol', 'price_date', 'open', 'high', 'low', 'close', 'adj_close', 'volume']
First few rows:
   fund_symbol  price_date   open   high    low  close  adj_close  volume
0         AAA  2020-09-09  25.10  25.12  25.07  25.07      24.85   17300
1         AAA  2020-09-10  25.06  25.07  25.05  25.07      24.85   23500
2         AAA  2020-09-11  25.04  25.05  25.02  25.03      24.81   33400
3         AAA  2020-09-14  25.01  25.06  25.01  25.02      24.80   13100
4         AAA  2020-09-15  25.02  25.03  25.01  25.01      24.79   12100
Please update col_mapping with correct column names from the printed column list above.
Error: Too few required columns available. Exiting.
Visualizations complete. Check for saved files: risk_vs_return.png, expense_ratio_by_type.png, returns_histogram.png, sharpe_ratio_boxplot.png, correlation_heatmap.png
